In [2]:
import sklearn
from sklearn import utils, neighbors, ensemble, svm, metrics

In [1]:
# Load the libraries
import os
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import seaborn as sns

In [5]:
#read_csv

labels=pd.read_csv("/Users/anaraquelpengelly/Desktop/MSC_health_data_science/term_2/machine_learning/project_malaria/Malaria_blood_image_classification/labels.csv")



In [9]:
#shuffle the df
random.seed(30)
labels = sklearn.utils.shuffle(labels)

In [10]:
labels.head()

,Unnamed: 0,0,infect_status
8090,58315,C39P4thinF_original_IMG_20150622_105335_cell_6...,1
26959,14862,C93P54ThinF_IMG_20150821_162641_cell_105.png,0
6785,51655,C182P143NThinF_IMG_20151201_172524_cell_204.png,1
26346,14177,C84P45ThinF_IMG_20150818_101146_cell_45.png,0
21850,9129,C227ThinF_IMG_20151112_135509_cell_173.png,0


In [37]:
labels.to_csv("/Users/anaraquelpengelly/Desktop/MSC_health_data_science/term_2/machine_learning/project_malaria/Malaria_blood_image_classification/shuffled_labels.csv")

## Splitting the training and test sets

Now we have a shuffled label df, we need to decide what the best way to split the training and test sets.
- 80% and 20% ? or something else? how do we determine the best way to do it?
- in any case we should have 50% of images that are infected and uninfected in the training set ( and the test set)

In [20]:
#sampleing labels for the training set:
training=labels.sample(frac=0.8, replace=False, random_state=30)

In [21]:
training.shape

(21946, 3)

In [22]:
training.head()

,Unnamed: 0,0,infect_status
8988,62154,C48P9thinF_IMG_20150721_161055_cell_195.png,1
26960,14863,C93P54ThinF_IMG_20150821_162641_cell_112.png,0
4945,43681,C170P131ThinF_IMG_20151119_120019_cell_244.png,1
4584,36096,C158P119ThinF_IMG_20151115_181136_cell_199.png,1
767,9586,C116P77ThinF_IMG_20150930_171558_cell_102.png,1


In [24]:
training["infect_status"].value_counts()

1    10973
0    10973
Name: infect_status, dtype: int64

In [26]:
training["0"].value_counts()

C85P46ThinF_IMG_20150820_110040_cell_185.png          1
C59P20thinF_IMG_20150803_113809_cell_39.png           1
C78P39ThinF_IMG_20150606_104106_cell_8.png            1
C39P4thinF_original_IMG_20150622_114122_cell_1.png    1
C161P122ThinF_IMG_20151116_101403_cell_101.png        1
                                                     ..
C144P105ThinF_IMG_20151015_160529_cell_170.png        1
C208ThinF_IMG_20151029_155827_cell_23.png             1
C133P94ThinF_IMG_20151004_154351_cell_94.png          1
C48P9thinF_IMG_20150721_161055_cell_195.png           1
C180P141NThinF_IMG_20151201_165659_cell_13.png        1
Name: 0, Length: 21946, dtype: int64

In [35]:
#sampleing lables for the test set: 
#Identify what values are in labels and not in training:
# %%timeit
diff=set(labels["0"]).difference(training["0"])
where_diff =labels["0"].isin(diff)
#slice labels and add create df
test=labels[where_diff]
test.shape

(5487, 3)

In [31]:
test.head()

,Unnamed: 0,0,infect_status
2868,20015,C132P93ThinF_IMG_20151004_152505_cell_135.png,1
22481,9842,C236ThinF_IMG_20151127_102635_cell_26.png,0
1506,11453,C119P80ThinF_IMG_20151002_123911_cell_139.png,1
4666,36762,C159P120ThinF_IMG_20151115_190642_cell_200.png,1
7459,52916,C184P145ThinF_IMG_20151203_104153_cell_164.png,1


In [33]:
test["infect_status"].value_counts()

1    2749
0    2738
Name: infect_status, dtype: int64

In [36]:
#now save both as csvs
path="/Users/anaraquelpengelly/Desktop/MSC_health_data_science/term_2/machine_learning/project_malaria/Malaria_blood_image_classification/"
training.to_csv((path+"training_labels.csv"))
test.to_csv((path+"test_labels.csv"))

In [41]:
#Just to check that thecsvs look fine 
a=pd.read_csv((path+"training_labels.csv"))
a.shape

(21946, 4)

In [42]:
b=pd.read_csv((path+"test_labels.csv"))
b.shape

(5487, 4)

In [43]:
c=pd.read_csv((path+"shuffled_labels.csv"))
c.shape

(27433, 4)

All looks good ecscept that we now have tw extra useless columns! 

Now we need to put all pictures in one folder and then create a function to grab from that folder the training and the test images.

## Split the training set in cross validation sets...